In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
from torchvision import datasets, models, transforms
import time
import os
from efficientnet.model import EfficientNet


In [3]:
#hyperparameters:
use_gpu = torch.cuda.is_available()
in_channel=3
class_num=2
lr=0.001
batch_size=32
data_dir = 'dataset_genero'
num_epochs=10
input_size = 224
momentum = 0.9
net_name = 'efficientnet-b0'

In [4]:
def loaddata(data_dir, batch_size, set_name, shuffle):
    data_transforms = {
        'train': transforms.Compose([
            transforms.Resize(input_size),
            transforms.CenterCrop(input_size),
            #transforms.RandomAffine(degrees=0, translate=(0.05, 0.05)),
            #transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ]),
        'test': transforms.Compose([
            transforms.Resize(input_size),
            transforms.CenterCrop(input_size),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ]),
    }

    image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x]) for x in [set_name]}
    # num_workers=0 if CPU else =1
    dataset_loaders = {x: torch.utils.data.DataLoader(image_datasets[x],
                                                      batch_size=batch_size,
                                                      shuffle=shuffle, num_workers=1) for x in [set_name]}
    data_set_sizes = len(image_datasets[set_name])
    return dataset_loaders, data_set_sizes

In [5]:
def train_model(model_ft, criterion, optimizer, lr_scheduler, num_epochs=50):
    train_loss = []
    since = time.time()
    best_model_wts = model_ft.state_dict()
    best_acc = 0.0
    model_ft.train(True)
    for epoch in range(num_epochs):
        dset_loaders, dset_sizes = loaddata(data_dir=data_dir, batch_size=batch_size, set_name='train', shuffle=True)
        print('Data Size', dset_sizes)
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)
        optimizer = lr_scheduler(optimizer, epoch)

        running_loss = 0.0
        running_corrects = 0
        count = 0

        for data in dset_loaders['train']:
            inputs, labels = data
            labels = torch.squeeze(labels.type(torch.LongTensor))
            if use_gpu:
                inputs, labels = Variable(inputs.cuda()), Variable(labels.cuda())
            else:
                inputs, labels = Variable(inputs), Variable(labels)

            outputs = model_ft(inputs)
            loss = criterion(outputs, labels)
            _, preds = torch.max(outputs.data, 1)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            count += 1
            if count % 30 == 0 or outputs.size()[0] < batch_size:
                print('Epoch:{}: loss:{:.3f}'.format(epoch, loss.item()))
                train_loss.append(loss.item())

            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)

        epoch_loss = running_loss / dset_sizes
        epoch_acc = running_corrects.double() / dset_sizes

        print('Loss: {:.4f} Acc: {:.4f}'.format(
            epoch_loss, epoch_acc))

        if epoch_acc > best_acc:
            best_acc = epoch_acc
            best_model_wts = model_ft.state_dict()
        if epoch_acc > 0.999:
            break

    # save best model
    save_dir = data_dir + '/model'
    model_ft.load_state_dict(best_model_wts)
    model_out_path = save_dir + "/" + net_name + '.pth'
    torch.save(model_ft, model_out_path)

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))

    return train_loss, best_model_wts

In [6]:
def test_model(model, criterion):
    model.eval()
    running_loss = 0.0
    running_corrects = 0
    cont = 0
    outPre = []
    outLabel = []
    dset_loaders, dset_sizes = loaddata(data_dir=data_dir, batch_size=16, set_name='test', shuffle=False)
    for data in dset_loaders['test']:
        inputs, labels = data
        labels = torch.squeeze(labels.type(torch.LongTensor))
        inputs, labels = Variable(inputs.cuda()), Variable(labels.cuda())
        outputs = model(inputs)
        _, preds = torch.max(outputs.data, 1)
        loss = criterion(outputs, labels)
        if cont == 0:
            outPre = outputs.data.cpu()
            outLabel = labels.data.cpu()
        else:
            outPre = torch.cat((outPre, outputs.data.cpu()), 0)
            outLabel = torch.cat((outLabel, labels.data.cpu()), 0)
        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)
        cont += 1
    print('Loss: {:.4f} Acc: {:.4f}'.format(running_loss / dset_sizes,
                                            running_corrects.double() / dset_sizes))


In [7]:
def exp_lr_scheduler(optimizer, epoch, init_lr=0.01, lr_decay_epoch=10):
    """Decay learning rate by a f#            model_out_path ="./model/W_epoch_{}.pth".format(epoch)
#            torch.save(model_W, model_out_path) actor of 0.1 every lr_decay_epoch epochs."""
    lr = init_lr * (0.8**(epoch // lr_decay_epoch))
    print('LR is set to {}'.format(lr))
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

    return optimizer

In [8]:
# train
pth_map = {
    'efficientnet-b0': 'efficientnet-b0-355c32eb.pth',
    'efficientnet-b1': 'efficientnet-b1-f1951068.pth',
    'efficientnet-b2': 'efficientnet-b2-8bb594d6.pth',
    'efficientnet-b3': 'efficientnet-b3-5fb5a3c3.pth',
    'efficientnet-b4': 'efficientnet-b4-6ed6700e.pth',
    'efficientnet-b5': 'efficientnet-b5-b6417697.pth',
    'efficientnet-b6': 'efficientnet-b6-c76e70fd.pth',
    'efficientnet-b7': 'efficientnet-b7-dcc49843.pth',
}


In [9]:
# model_ft = EfficientNet.from_pretrained('efficientnet-b0')
net_name="efficientnet-b0"
model_ft = EfficientNet.from_name(net_name)
net_weight = 'efficientnet/' + pth_map[net_name]
state_dict = torch.load(net_weight)
model_ft.load_state_dict(state_dict)


<All keys matched successfully>

In [10]:

num_ftrs = model_ft._fc.in_features
model_ft._fc = nn.Linear(num_ftrs, class_num)

criterion = nn.CrossEntropyLoss()
if use_gpu:
    model_ft = model_ft.cuda()
    criterion = criterion.cuda()

optimizer = optim.SGD((model_ft.parameters()), lr=lr,
                      momentum=momentum, weight_decay=0.0004)

train_loss, best_model_wts = train_model(model_ft, criterion, optimizer, exp_lr_scheduler, num_epochs=num_epochs)



Data Size 24621
Epoch 0/9
----------
LR is set to 0.01
Epoch:0: loss:0.676
Epoch:0: loss:0.498
Epoch:0: loss:0.498
Epoch:0: loss:0.473
Epoch:0: loss:0.526
Epoch:0: loss:0.318
Epoch:0: loss:0.510
Epoch:0: loss:0.535
Epoch:0: loss:0.390
Epoch:0: loss:0.644
Epoch:0: loss:0.522
Epoch:0: loss:0.556
Epoch:0: loss:0.679
Epoch:0: loss:0.427
Epoch:0: loss:0.391
Epoch:0: loss:0.444
Epoch:0: loss:0.332
Epoch:0: loss:0.375
Epoch:0: loss:0.412
Epoch:0: loss:0.495
Epoch:0: loss:0.328
Epoch:0: loss:0.602
Epoch:0: loss:0.551
Epoch:0: loss:0.359
Epoch:0: loss:0.390
Epoch:0: loss:0.559
Loss: 0.4990 Acc: 0.7522
Data Size 24621
Epoch 1/9
----------
LR is set to 0.01
Epoch:1: loss:0.318
Epoch:1: loss:0.355
Epoch:1: loss:0.465
Epoch:1: loss:0.382
Epoch:1: loss:0.438
Epoch:1: loss:0.252
Epoch:1: loss:0.272
Epoch:1: loss:0.398
Epoch:1: loss:0.378
Epoch:1: loss:0.299
Epoch:1: loss:0.390
Epoch:1: loss:0.232
Epoch:1: loss:0.426
Epoch:1: loss:0.171
Epoch:1: loss:0.460
Epoch:1: loss:0.409
Epoch:1: loss:0.354
Epoch

In [18]:
from torchvision import datasets, models, transforms
import torch
import torch.nn as nn
import torch.optim as optim

PATH = "resultado/model/efficientnet-b0.pth"

# Load
model = torch.load(PATH)
model.eval()

# Create the preprocessing transformation here
transform = transforms.ToTensor()

def loaddata(data_dir, batch_size, set_name, shuffle):
    data_transforms = {
        'train': transforms.Compose([
            transforms.Resize(input_size),
            transforms.CenterCrop(input_size),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ]),
        'test': transforms.Compose([
            transforms.Resize(input_size),
            transforms.CenterCrop(input_size),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ]),
    }

from PIL import Image
# Preprocess image
tfms = transforms.Compose([transforms.Resize(224), transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),])
img = tfms(Image.open('images/tijuana_aer.jpeg')).unsqueeze(0)
#img = tfms(Image.open('dataset_genero/test/m/61d8cf0ad0dd825c2753b614.jpg')).unsqueeze(0)
img= img.cuda()
# Load ImageNet class names
labels_map = ["f","m"]

# Classify
model.eval()
with torch.no_grad():
    outputs = model(img)

# Print predictions
print(torch.softmax(outputs, dim=1))
print('-----')

for idx in torch.topk(outputs, k=1).indices.squeeze(0).tolist():
    prob = torch.softmax(outputs, dim=1)[0, idx].item()
    print('{label:<75} ({p:.2f}%)'.format(label=labels_map[idx], p=prob*100))

tensor([[1.0000e+00, 2.4121e-11]], device='cuda:0')
-----
f                                                                           (100.00%)
